In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import os
path = os.getcwd()

path_mm = os.path.dirname(path)
import sys
sys.path.append(path_mm)


# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from importlib import reload
import gc
from pyoperators import *
from pysimulators import *


from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")

# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

rc('figure',figsize=(20,10))
rc('font',size=12)
rc('image', cmap='jet')


nside = 512
stokes = ['I', 'Q', 'U']

In [ ]:
### Simplistic proxy for QUBIC Patch
ra_center = 0 # degrees
dec_center = -57 # degrees
radius = 20. # degrees
center = qubic.equ2gal(ra_center, dec_center)

veccenter = hp.ang2vec(center[0], center[1], lonlat=True)
seenpix = hp.query_disc(nside, veccenter, np.radians(radius))

mask = np.zeros(12*nside**2, dtype=bool)
mask[seenpix] = True

hp.mollview(mask, sub=(1,2,1))
hp.gnomview(mask, rot=center, reso=15, sub=(1,2,2))


In [ ]:
### We define foregrounds model
sky_dust = pysm3.Sky(nside, preset_strings=['d0'])
sky_cmb = pysm3.Sky(nside, preset_strings=['c1'])
sky = pysm3.Sky(nside, preset_strings=['c1', 'd0'])

### Return frequency map according to nu and sky model
def get_pySM_maps(sky, nu):
    return np.array(sky.get_emission(nu * u.GHz, None).T * utils.bandpass_unit_conversion(nu*u.GHz, None, u.uK_CMB))

### We compute components, we should take Planck components for real data
nu_ref = 150
C = HealpixConvolutionGaussianOperator(fwhm=np.deg2rad(0.1))

mydust = C(get_pySM_maps(sky_dust, nu_ref))
mycmb = C(get_pySM_maps(sky_cmb, nu_ref))
mysky = C(get_pySM_maps(sky, nu_ref))

mydustQ = mydust.copy()
mydustQ[~mask,:] = hp.UNSEEN 
mycmbQ = mycmb.copy()
mycmbQ[~mask,:] = hp.UNSEEN 
myskyQ = mysky.copy()
myskyQ[~mask,:] = hp.UNSEEN 



In [ ]:
for i in range(len(stokes)):
    if i == 0:
        mini = -300
        maxi = 300
    else:
        mini = -5
        maxi = 5
    figure()
    suptitle('Stokes {}'.format(stokes[i]), fontsize=30)
    hp.mollview(mydust[:,i], sub=(2,3,1), title='Dust {} @ {} GHz'.format(stokes[i], nu_ref), min=mini, max=maxi, unit='μK')
    hp.mollview(mycmb[:,i], sub=(2,3,2), title='CMB {} @ {} GHz'.format(stokes[i], nu_ref), min=mini, max=maxi, unit='μK')
    hp.mollview(mysky[:,i], sub=(2,3,3), title='Both {} @ {} GHz'.format(stokes[i], nu_ref), min=mini, max=maxi, unit='μK')

    hp.gnomview(mydustQ[:,i], sub=(2,3,4), reso=15, rot=center, title='Dust {} @ {} GHz'.format(stokes[i], nu_ref), min=mini, max=maxi, unit='μK')
    hp.gnomview(mycmbQ[:,i], sub=(2,3,5), reso=15, rot=center, title='CMB {} @ {} GHz'.format(stokes[i], nu_ref), min=mini, max=maxi, unit='μK')
    hp.gnomview(myskyQ[:,i], sub=(2,3,6), reso=15, rot=center, title='Both {} @ {} GHz'.format(stokes[i], nu_ref), min=mini, max=maxi, unit='μK')



# Now polarization display
We would like to perform the following displays:
1. Displaying polarization vectors on top of polarization amplitude
2. Displaying drapery patterns over the maps
3. Displaying stacked polarization hot & cold spots in order to reveal the E/B modes

# Polarization amplitude

In [ ]:
def getP(maps, mask):
    P = np.sqrt(maps[:,1]**2 + maps[:,2]**2) / (2.7e6+maps[:,0])
    PQ = P.copy()
    PQ[~mask] = hp.UNSEEN
    return P, PQ

Pdust, PdustQ = getP(mydust, mask)
Pcmb, PcmbQ = getP(mycmb, mask)
Psky, PskyQ = getP(mysky, mask)

mini = 0
maxi = 1e-5
hp.mollview(Pdust, sub=(2,3,1), title='Dust P @ {} GHz'.format(nu_ref), min=mini, max=maxi)
hp.mollview(Pcmb, sub=(2,3,2), title='CMB P @ {} GHz'.format(nu_ref), min=mini, max=maxi)
hp.mollview(Psky, sub=(2,3,3), title='Both P @ {} GHz'.format(nu_ref), min=mini, max=maxi)

hp.gnomview(PdustQ, sub=(2,3,4), reso=15, rot=center, title='Dust @ {} GHz'.format(nu_ref), min=mini, max=maxi)
hp.gnomview(PcmbQ, sub=(2,3,5), reso=15, rot=center, title='CMB @ {} GHz'.format(nu_ref), min=mini, max=maxi)
hp.gnomview(PskyQ, sub=(2,3,6), reso=15, rot=center, title='Both @ {} GHz'.format(nu_ref), min=mini, max=maxi)


# Drapery patterns
https://github.com/healpy/healpy/pull/617#issue-434041253

In [ ]:
from matplotlib.cm import get_cmap
from matplotlib.colors import ListedColormap

cmap_colors = get_cmap('binary', 256)(np.linspace(0, 1, 256))
cmap_colors[..., 3] = 0.4  # Make colormap partially transparent
cmap = ListedColormap(cmap_colors)

def licmaps(maps, mask, sm=5., smlic=0.5, display=None, qubic=True, sub=None, maptoplot='P'):
    I, Q, U = hp.smoothing(maps.T.copy(), deg2rad(sm))
    lic = hp.line_integral_convolution(Q, U)
    lic = hp.smoothing(lic, deg2rad(smlic))

    if maptoplot == 'P':
        # Pol Fraction ???
        P = np.log(1 + np.sqrt(Q**2 + U**2) * 100)
    elif maptoplot == 'myP':
        P = np.sqrt(Q**2 + U**2) / np.abs(I)
    elif maptoplot == 'I':
        P = I.copy()

    # Apply masks
    PQ = P.copy()
    P[~mask] = hp.UNSEEN
    licQ = lic.copy()
    licQ[~mask] = hp.UNSEEN

    if display is not None:
        if qubic:
            mP = hp.gnomview(PQ, rot=center, reso=15, cmap='jet', cbar=True, return_projected_map=True)
            mlic = hp.gnomview(licQ, rot=center, reso=15, cmap=cmap, cbar=False, reuse_axes=True, title=display, sub=sub, return_projected_map=True)
        else:
            hp.mollview(P, cmap='jet', cbar=True)
            hp.mollview(lic, cmap=cmap, cbar=False, reuse_axes=True, title=display, sub=sub)
            
    return P, lic, PQ, licQ





In [ ]:
_,_,_,_ = licmaps(mydust, mask, display='Dust', qubic=True, sm=2, maptoplot='P', smlic=0.3) 

In [ ]:
_,_,_,_ = licmaps(mycmb, mask, display='CMB', qubic=True, sm=2, maptoplot='P', smlic=0.3) 

In [ ]:
_,_,_,_ = licmaps(mysky, mask, display='Both', qubic=True, sm=2, maptoplot='P', smlic=0.3) 

# Polarization stacking

In [ ]:
### Find extremas in the smoothed map
ns_used = nside

map2use = hp.ud_grade(hp.smoothing(mycmb[:,0], deg2rad(1.)), ns_used)
maphs, minpix, maxpix = hp.hotspots(map2use)
hp.mollview(maphs, sub=(1,2,1), title='Hot: {}   - Cold: {}'.format(len(minpix), len(maxpix)))

In [ ]:
def show_gnom_around_pix(themap, pix, reso, sub=None, no_plot=True, xsize=None):
    ns = hp.npix2nside(len(themap))
    rot = hp.pix2ang(ns, pix, lonlat=True)
    mm = hp.gnomview(themap, rot=rot, reso=reso, sub=sub, return_projected_map=True, no_plot=no_plot, xsize=xsize)
    return mm

def iterate_allpix(mapmax, mapQ, mapU, maxpixels, npixmap=100):
    reso = 2.5*60/(npixmap/2)
    mm = np.zeros((npixmap, npixmap))
    mmQ = np.zeros((npixmap, npixmap))
    mmU = np.zeros((npixmap, npixmap))

    ### The big loop
    bar = qubic.progress_bar(len(maxpixels))
    for i in range(len(maxpixels)):
        bar.update()
        mm += show_gnom_around_pix(mapmax, maxpixels[i], reso, xsize=npixmap) / nb
        mmQ += show_gnom_around_pix(mapQ, maxpixels[i], reso, xsize=npixmap) / nb
        mmU += show_gnom_around_pix(mapU, maxpixels[i], reso, xsize=npixmap) / nb

    ### Remove offsets
    newQ = mmQ - np.median(mmQ)
    newU = mmU - np.median(mmU)

    ### Transform into the radial Q & U
    x0 = (np.arange(npixmap)-npixmap/2+0.5) * reso / 60
    xx, yy = np.meshgrid(x0, x0)
    theta = sqrt(xx**2+yy**2)
    phi = (np.pi-arctan2(yy,xx))
    Qr = -newQ * cos(2*phi) - newU * sin(2*phi)
    Ur = newQ * sin(2*phi) - newU * cos(2*phi)
    
    return mm, mmQ, mmU, Qr, Ur

npixmap = 100
reso = 2.5*60/(npixmap/2)
nn = 3

themap = hp.ud_grade(mycmb.T, ns_used).T
mypixels = maxpix.copy()

show_gnom_around_pix(themap[:,0], mypixels[0], reso, xsize=npixmap, no_plot=False)
show()


In [ ]:
### Stacking on maxima
print('Doing Maxima')
mmpos, mmQpos, mmUpos, Qrpos, Urpos = iterate_allpix(themap[:,0], themap[:,1], themap[:,2], maxpix, npixmap=100)

### Stacking on minima
print('Doing Minima')
mmneg, mmQneg, mmUneg, Qrneg, Urneg = iterate_allpix(themap[:,0], themap[:,1], themap[:,2], minpix, npixmap=100)

In [ ]:
import skimage.measure #(conda install scikit-image)


def plot_vector_resample(xx, yy, Q, U, Z, nrs=5):
    new_xx = skimage.measure.block_reduce(xx, (nrs,nrs), np.mean)
    new_yy = skimage.measure.block_reduce(yy, (nrs,nrs), np.mean)
    new_mmQ = skimage.measure.block_reduce(Q, (nrs,nrs), np.mean)
    new_mmU = skimage.measure.block_reduce(U, (nrs,nrs), np.mean)
    new_Z = skimage.measure.block_reduce(Z, (nrs,nrs), np.mean)
    
    gamma = -0.5 * arctan2(-new_mmU,-new_mmQ)
    xP = new_Z * cos(gamma)
    yP = new_Z * sin(gamma)

    quiver(new_xx,new_yy,xP,yP, width=0.005, headwidth=0, headlength=0, headaxislength=0, pivot='mid')



In [ ]:
subplot(2,3,1)
imshow(mmpos, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], 
       cmap = 'jet', origin='lower', vmin=-140, vmax=140)
xlabel('Degrees')
ylabel('Degrees')
title('Stokes I')
colorbar()
grid()

subplot(2,3,2)
imshow(mmQpos, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', 
       cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
title('Stokes Q')
colorbar()
grid()

subplot(2,3,3)
imshow(mmUpos, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', 
       cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
title('Stokes U')
colorbar()
grid()

subplot(2,2,3)
imshow(Qrpos, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', 
       cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
colorbar()
title('Qr')
grid()
Prpos = sqrt(Qrpos**2 + Urpos**2)
plot_vector_resample(xx, yy, mmQpos, mmUpos, Prpos, nrs=4)

subplot(2,2,4)
imshow(Urpos, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
colorbar()
title('Ur')
grid()

tight_layout()

In [ ]:
subplot(2,3,1)
imshow(mmneg, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], 
       cmap = 'jet', origin='lower', vmin=-140, vmax=140)
xlabel('Degrees')
ylabel('Degrees')
title('Stokes I')
colorbar()
grid()

subplot(2,3,2)
imshow(mmQneg, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', 
       cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
title('Stokes Q')
colorbar()
grid()

subplot(2,3,3)
imshow(mmUneg, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', 
       cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
title('Stokes U')
colorbar()
grid()

subplot(2,2,3)
imshow(Qrneg, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', 
       cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
colorbar()
title('Qr')
grid()
Prneg = sqrt(Qrneg**2 + Urneg**2)
plot_vector_resample(xx, yy, mmQneg, mmUneg, Prneg, nrs=4)

subplot(2,2,4)
imshow(Urneg, extent=[-npixmap/2*reso/60, npixmap/2*reso/60, -npixmap/2*reso/60, npixmap/2*reso/60], origin='lower', cmap='jet', vmin=-0.5, vmax=0.5)
xlabel('Degrees')
ylabel('Degrees')
colorbar()
title('Ur')
grid()

tight_layout()

# Now for the QUBIC Patch

In [ ]:
### Find extremas in the smoothed map
ns_used = nside
map2use = hp.ud_grade(hp.smoothing(mycmb[:,0], deg2rad(1.)), ns_used)
maphs, minpix, maxpix = hp.hotspots(map2use)

### but we only keep thos in the QUBIC mask
maphs[~mask] = hp.UNSEEN
minpix = minpix[mask[minpix] == True]
maxpix = maxpix[mask[maxpix] == True]

hp.mollview(maphs, sub=(1,2,1), title='Hot: {}   - Cold: {}'.format(len(minpix), len(maxpix)))
hp.gnomview(maphs, sub=(1,2,2), rot=center, reso=20, title='Hot: {}   - Cold: {}'.format(len(minpix), len(maxpix)))

In [ ]:
# reso = 2.5*60/100
# nn = 3
# mm = np.zeros((200, 200))
# mmQ = np.zeros((200, 200))
# mmU = np.zeros((200, 200))
# nb = len(maxpix)#*0+300
# for i in range(nb):
#     if (i%100) == 0: print(i)
#     mm += show_gnom_around_pix(mycmb[:,0], maxpix[i], reso) / nb
#     mmQ += show_gnom_around_pix(mycmb[:,1], maxpix[i], reso) / nb
#     mmU += show_gnom_around_pix(mycmb[:,2], maxpix[i], reso) / nb


In [ ]:
# subplot(2,4,1)
# imshow(mm, extent=[-100*reso/60, 100*reso/60, -100*reso/60, 100*reso/60])
# xlabel('Degrees')
# ylabel('Degrees')
# title('Stokes I')
# colorbar()
# grid()

# subplot(2,4,2)
# imshow(mmQ, extent=[-100*reso/60, 100*reso/60, -100*reso/60, 100*reso/60])
# xlabel('Degrees')
# ylabel('Degrees')
# title('Stokes Q')
# colorbar()
# grid()

# subplot(2,4,3)
# imshow(mmU, extent=[-100*reso/60, 100*reso/60, -100*reso/60, 100*reso/60])
# xlabel('Degrees')
# ylabel('Degrees')
# title('Stokes U')
# colorbar()
# grid()

# subplot(2,4,4)
# imshow(np.sqrt(mmQ**2 + mmU**2), extent=[-100*reso/60, 100*reso/60, -100*reso/60, 100*reso/60])
# xlabel('Degrees')
# ylabel('Degrees')
# title('$P=\sqrt{Q^2+U^2}$')
# colorbar()
# grid()

# tight_layout()